# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

N. Martin  ->  N. Martin  |  ['N. Martin']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
T. Henning  ->  T. Henning  |  ['T. Henning']
Arxiv has 42 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2502.09792


extracting tarball to tmp_2502.09792...

 done.


N. Martin  ->  N. Martin  |  ['N. Martin']


...

extracting tarball to tmp_2502.10015...

 done.
Retrieving document from  https://arxiv.org/e-print/2502.10271


extracting tarball to tmp_2502.10271...

 done.


Found 108 bibliographic references in tmp_2502.10271/main.bbl.
Issues with the citations
syntax error in line 25: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.09792-b31b1b.svg)](https://arxiv.org/abs/2502.09792) | **Deep in the Fields of the Andromeda Halo: Discovery of the Pegasus VII dwarf galaxy in UNIONS**  |
|| S. E. T. Smith, et al. -- incl., <mark>N. Martin</mark> |
|*Appeared on*| *2025-02-18*|
|*Comments*| *13 pages, 5 figures, accepted for publication in ApJ*|
|**Abstract**|            We present the newly discovered dwarf galaxy Pegasus VII (Peg VII), a member of the M31 sub-group which has been uncovered in the $ri$ photometric catalogs from the Ultraviolet Near-Infrared Optical Northern Survey and confirmed with follow-up imaging from both the Canada-France-Hawaii Telescope and the Gemini-North Telescope. This system has an absolute $V$-band magnitude of $-5.7 \pm 0.2$ mag and a physical half-light radius of $177^{+36}_{-34}$ pc, which is characteristic of dynamically-confirmed Milky Way satellite dwarf galaxies and about 5 times more extended than the most extended M31 globular clusters. Peg VII lies at a three-dimensional separation from M31 of $331^{+15}_{-4}$ kpc and a significant elongation ($\epsilon \sim 0.5$) towards the projected direction of M31 could be indicative of a past tidal interaction, but additional investigation into the orbit, star formation history, and whether any gas remains in the galaxy is needed to better understand the evolution of Peg VII.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.10271-b31b1b.svg)](https://arxiv.org/abs/2502.10271) | **MICONIC: JWST Unveils Shocked Hot Core Chemistry in the Western Nucleus of Arp 220**  |
|| V. Buiten, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-02-18*|
|*Comments*| *Submitted to A&A*|
|**Abstract**|            We present full 3-28 micron JWST MIRI/MRS and NIRSpec/IFU spectra of the western nucleus of Arp 220, the nearest ultraluminous infrared galaxy. This nucleus has long been suggested to possibly host an embedded Compton-thick AGN. Millimeter observations of the dust continuum suggest the presence of a distinct 20 pc core with a dust temperature of $T_\mathrm{d} \gtrsim 500~\mathrm{K}$, in addition to a 100 pc circumnuclear starburst disk. However, unambiguously identifying the nature of this core is challenging, due to the immense obscuration, the nuclear starburst activity, and the nearby eastern nucleus. With the JWST integral field spectrographs, we can, for the first time, separate the two nuclei across this full wavelength range, revealing a wealth of molecular absorption features towards the western nucleus. We analyse the rovibrational bands detected at 4-22 micron, deriving column densities and rotational temperatures for 10 distinct species. Optically thick features of C$_2$H$_2$, HCN and HNC suggest that this molecular gas is hidden behind a curtain of cooler dust, and indicate that the column densities of C$_2$H$_2$ and HCN are an order of magnitude higher than previously derived from Spitzer observations. We identify a warm HCN component with rotational temperature $T_\mathrm{rot} = 330~\mathrm{K}$, which we associate with radiative excitation by the hot inner nucleus. We propose a geometry where the detected molecular gas is located in the inner regions of the starburst disk, directly surrounding the hot 20 parsec core. The chemical footprint of the western nucleus is reminiscent of that of hot cores, with additional evidence for shocks. No evidence for the presence of an AGN in the form of X-ray-driven chemistry or extreme excitation is found.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.10015-b31b1b.svg)](https://arxiv.org/abs/2502.10015) | **An Ultra-Fast Image Simulation Technique with Spatially Variable Point Spread Functions**  |
|| Z. Bai, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-02-18*|
|*Comments*| *To be published in the AJ. Comments are welcome*|
|**Abstract**|            Simulated images are essential in algorithm development and instrument testing for optical telescopes. During real observations, images obtained by optical telescopes are affected by spatially variable point spread functions (PSFs), a crucial effect requiring accurate simulation. Traditional methods segment images into patches, convolve patches with individual PSFs, and reassemble them as a whole image. Although widely used, these approaches suffer from slow convolution processes and reduced image fidelity due to abrupt PSF transitions between different patches. This paper introduces a novel method for generating simulated images with spatial continuously varying PSFs. Our approach firstly decomposes original images into PSF bases derived with the principal component analysis method. The entire image is then convolved with these PSF bases to create image bases. Finally, we multiply the coefficients of image bases with these image bases for each pixels and add the multiplication results along each pixel to obtain the final simulated image. Our method could generate high-fidelity simulated images with spatially variable PSFs without boundary artifacts. The method proposed in this paper significantly improves the speed of astronomical image simulation, potentially advancing observational astronomy and instrumental development.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2502.09792/./peg7-cmd-hess-new.png', 'tmp_2502.09792/./peg7-spatial-arrows-wedge.png', 'tmp_2502.09792/./peg7-HB-dist.png', 'tmp_2502.09792/./m31-field-ebv-rect.png']
copying  tmp_2502.09792/./peg7-cmd-hess-new.png to _build/html/
copying  tmp_2502.09792/./peg7-spatial-arrows-wedge.png to _build/html/
copying  tmp_2502.09792/./peg7-HB-dist.png to _build/html/
copying  tmp_2502.09792/./m31-field-ebv-rect.png to _build/html/
exported in  _build/html/2502.09792.md
    + _build/html/tmp_2502.09792/./peg7-cmd-hess-new.png
    + _build/html/tmp_2502.09792/./peg7-spatial-arrows-wedge.png
    + _build/html/tmp_2502.09792/./peg7-HB-dist.png
    + _build/html/tmp_2502.09792/./m31-field-ebv-rect.png
found figures ['tmp_2502.10271/./H2O_model_spec_compilation_loglam.png', 'tmp_2502.10271/./line_of_sight_geometry.png', 'tmp_2502.10271/./toy_model_continuum.png', 'tmp_2502.10271/./Arp220_WN_cartoon_v1.png']
copying  tmp_2502.10271/./H2O_model_spec_compilation_loglam.png to _build

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\chg}[1]{\textbf{#1}}$
$\newcommand{\warn}[1]{\textcolor{red}{\textbf{#1}}}$
$\newcommand{\plot}[1]{\textit{#1}:}$
$\newcommand{\code}[1]{\texttt{#1}}$
$\newcommand{\rh}{r_{\text{h}}}$
$\newcommand{\rhm}{r_{\text{h}}}$
$\newcommand{\ebv}{E(B-V)}$
$\newcommand{\Rv}{R_{V}}$
$\newcommand{\vh}{v_{\odot}}$
$\newcommand{\Msun}{\textup{M}_\odot}$
$\newcommand{\Lsun}{\textup{L}_\odot}$
$\newcommand{\sqdeg}{deg^2}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\met}{[Fe/H]}$
$\newcommand{\peg}{Peg VII}$</div>



<div id="title">

# Deep in the Fields of the Andromeda Halo: \\Discovery of the Pegasus VII dwarf galaxy in UNIONS

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2502.09792-b31b1b.svg)](https://arxiv.org/abs/2502.09792)<mark>Appeared on: 2025-02-18</mark> -  _13 pages, 5 figures, accepted for publication in ApJ_

</div>
<div id="authors">

S. E. T. Smith, et al. -- incl., <mark>N. Martin</mark>

</div>
<div id="abstract">

**Abstract:** We present the newly discovered dwarf galaxy Pegasus VII ( $\peg$ ), a member of the M31 sub-group which has been uncovered in the $ri$ photometric catalogs from the Ultraviolet Near-Infrared Optical Northern Survey and confirmed with follow-up imaging from both the Canada-France-Hawaii Telescope and the Gemini-North Telescope. This system has an absolute $V$ -band magnitude of $-5.7$ $\pm$ 0.2 mag and a physical half-light radius of 177 $^{+36}_{-34}$ pc, which is characteristic of dynamically-confirmed Milky Way satellite dwarf galaxies and about 5 times more extended than the most extended M31 globular clusters. $\peg$ lies at a three-dimensional separation from M31 of 331 $^{+15}_{-4}$ kpc and    a significant elongation ( $\epsilon \sim 0.5$ ) towards the projected direction of M31 could be indicative of a past tidal interaction, but additional investigation into the orbit, star formation history, and whether any gas remains in the galaxy is needed to better understand the evolution of $\peg$ .

</div>

<div id="div_fig1">

<img src="tmp_2502.09792/./peg7-cmd-hess-new.png" alt="Fig3" width="100%"/>

**Figure 3. -** _ Left_: CMD of stellar sources (black markers) within 2 half-light radii of the $\peg$ centroid. An old (10 Gyr), metal-poor ([Fe/H] = $-2$) PARSEC isochrone is overlaid in red, shifted to a distance of 750 kpc. The 5$\sigma$ point source depths ($g \sim 26.1$ and $r \sim 25.7$ mag) are indicated with a dashed black line. Black crosses show the median color and magnitude errors as a function of magnitude. Dashed orange outlines have been drawn to delineate where putative member stars of $\peg$ fall on the CMD, as described in Section \ref{subsec:stardist}.
    _ Centre_: This CMD shows sources which fall in an elliptical annulus whose inner radius is 3 half-light radii and has an area equal to the inner 2 half-light radius region, demonstrating the non-member source contamination present in the left panel.
    _ Right_: Smoothed Hess difference plot between on-target and field CMDs. Light regions indicate a high density of $\peg$ stars compared to field stars, while dark regions show deficits. The adopted isochrone is shown in red and we also shift the distance of the HB by 60 kpc (orange dashed lines) to indicate how the distance uncertainty is inferred from the difference CMD.
     (*fig:peg7-cmd*)

</div>
<div id="div_fig2">

<img src="tmp_2502.09792/./peg7-spatial-arrows-wedge.png" alt="Fig4.1" width="50%"/><img src="tmp_2502.09792/./peg7-HB-dist.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** _ Left_: Tangent plane-projected spatial distribution of cleaned stellar sources detected in Gemini-N/GMOS imaging about $\peg$. Black stars fall within the isochrone membership selection box defined in the left panel of Figure \ref{fig:peg7-cmd} while grey stars fall outside of this selection box. Concentric ellipses in yellow, orange, and red show the 1, 2, \& 3 $\times$\rh iso-density contours, respectively, of the best-fit stellar distribution model. The blue arrow indicates the tangent plane-projected direction towards M31 and the grey wedge shows the 1$\sigma$ uncertainties on the projected direction towards M31 stemming from the position angle of the major axis of $\peg$. The direction towards M31 is 18$^{+12}_{-9}$$\deg$ from the position angle of the major axis.
    _ Right_: Here we plot in large blue markers the tangent plane-projected spatial distribution of all suspected horizontal branch stars (defined by the HB selection box in the left panel of Figure \ref{fig:peg7-cmd}). We overlay black contours showing the smoothed density distribution of these same HB stars. We also plot small grey markers for all candidate member stars (same as black point in left panel of this figure), demonstrating how the HB distribution traces the same overdensity as the rest of the member stars.
     (*fig:peg7-spatial*)

</div>
<div id="div_fig3">

<img src="tmp_2502.09792/./m31-field-ebv-rect.png" alt="Fig5" width="100%"/>

**Figure 5. -** Tangent plane projection centered on M31, which is shown as the central black ellipse.
    Known galaxies in the vicinity of M31 are shown as black circles where their filled shade of blue indicates total absolute $V$-band magnitude in the range indicated in the color bar. The position of M33 (Triangulum) is marked with a white triangle. The three concentric, dashed black circles indicate projected radii of 100, 200, \& 300 kpc at the distance of M31. The dashed grey outline demarcates the PAndAS footprint while the UNIONS footprint in the south galactic cap is shown with a thick dashed green line, as noted in the legend.
    $\peg$ is indicated with a label as well as a large black circle and lies in front of the tangent plane, at a total three-dimensional distance of $\sim$330 kpc from M31.
    All known galaxies in this field of view with projected M31 separations greater than 200 kpc are labelled.
    The greyscale background shows the dust maps from \citet{Schlegel98} where the color bar has a maximum of $E(B-V) = 1$ to show the features around the M31 satellites, though the top band reaches much greater extinctions towards the galactic midplane (indicated by the dashed white line). Triangulum II and Segue 2 are Milky Way satellites in the foreground of this field of view. (*fig:m31-field*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2502.09792"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thought}[1]{\textcolor{red}{#1}}$
$\newcommand{\PvdW}[1]{\textcolor{purple}{[PvdW: #1]}}$
$\newcommand{\pvdw}[1]{\textcolor{purple}{[PvdW: #1]}}$</div>



<div id="title">

# MICONIC: JWST Unveils Shocked Hot Core Chemistry in the Western Nucleus of Arp 220

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2502.10271-b31b1b.svg)](https://arxiv.org/abs/2502.10271)<mark>Appeared on: 2025-02-18</mark> -  _Submitted to A&A_

</div>
<div id="authors">

V. Buiten, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** We present full $\qtyrange{3}{28}{\micron}$ JWST MIRI/MRS and NIRSpec/IFU spectra of the western nucleus of Arp 220, the nearest ultraluminous infrared galaxy. This nucleus has long been suggested to possibly host an embedded Compton-thick AGN. Millimeter observations of the dust continuum suggest the presence of a distinct $\qty{20}{\parsec}$ core with a dust temperature of $T_\mathrm{d} \gtrsim \qty{500}{\kelvin}$ , in addition to a $\qty{100}{\parsec}$ circumnuclear starburst disk. However, unambiguously identifying the nature of this core is challenging, due to the immense obscuration, the nuclear starburst activity, and the nearby eastern nucleus. With the JWST integral field spectrographs, we can, for the first time, separate the two nuclei across this full wavelength range, revealing a wealth of molecular absorption features towards the western nucleus. We analyse the rovibrational bands detected at $\qtyrange[range-units=single]{4}{22}{\micron}$ , deriving column densities and rotational temperatures for 10 distinct species. Optically thick features of $\ce{C2H2}$ , $\ce{HCN}$ and $\ce{HNC}$ suggest that this molecular gas is hidden behind a curtain of cooler dust, and indicate that the column densities of $\ce{C2H2}$ and $\ce{HCN}$ are an order of magnitude higher than previously derived from _Spitzer_ observations. We identify a warm $\ce{HCN}$ component with rotational temperature $T_\mathrm{rot} = \qty{330}{\kelvin}$ , which we associate with radiative excitation by the hot inner nucleus. We propose a geometry where the detected molecular gas is located in the inner regions of the starburst disk, directly surrounding the hot $\qty{20}{\parsec}$ core. The chemical footprint of the western nucleus is reminiscent of that of hot cores, with additional evidence for shocks. No evidence for the presence of an AGN in the form of X-ray-driven chemistry or extreme excitation is found.

</div>

<div id="div_fig1">

<img src="tmp_2502.10271/./H2O_model_spec_compilation_loglam.png" alt="Fig8" width="100%"/>

**Figure 8. -** The spectrum around \SI{6}{\micron} and the best-fit \ce{H2O} model for $f_\mathrm{bg} = 0.35$. The top panel shows the absolute flux spectrum and continuum model. The middle panel shows the continuum-extracted spectrum. Strong \ce{H2O} lines are labelled by their upper- and lower-level quantum numbers as $J_{u}, K_{a,u}, K_{c,u} - J_l, K_{a,l}, K_{c,l}$. Several strong emission lines and absorption lines of \ce{C2H} are labelled as well. The pink shaded area indicates the spectral region considered in the fit. The bottom panel shows the residuals on the continuum-extracted spectrum; the grey shaded area indicates their 68\% interval. (*fig:H2O_model_spec*)

</div>
<div id="div_fig2">

<img src="tmp_2502.10271/./line_of_sight_geometry.png" alt="Fig2.1" width="50%"/><img src="tmp_2502.10271/./toy_model_continuum.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Top: schematic illustration of the proposed line-of-sight geometry. The molecular gas is embedded in cool dust that produces significant foreground emission, filling in the features produced by absorption of hot background photons. Bottom: expected continuum spectrum for a toy model consisting of a \SI{1000}{\kelvin} background blackbody and a \SI{100}{\kelvin} foreground modified blackbody. (*fig:geometry_los_illustration*)

</div>
<div id="div_fig3">

<img src="tmp_2502.10271/./Arp220_WN_cartoon_v1.png" alt="Fig5" width="100%"/>

**Figure 5. -** Schematic illustration of the proposed geometry of the WN along the line of sight. Following the interpretation of \citet{Sakamoto2021b}, we assume that the south side of the disk is facing us with an inclination of $i \approx \ang{60}$, and that the biconical outflow is normal to the disk. The direction of the tilt is debated \citep[][Van der Werf et al. in prep]{Scoville1998}, but we note that it does not affect the interpretation of the absorption bands in this work. (*fig:WN_geometry_cartoon*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2502.10271"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

336  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
